# THE BURMAN TRAVELING SALESMAN PROBLEM (TSP) 

The travelling salesman problem (TSP) asks the following question: Given a list of cities and the distances between each pair of cities, what is the shortest possible route that visits each city exactly once and returns to the origin city? It is an NP-hard problem in combinatorial optimization, important in operations research and theoretical computer science [Wikipedia].

The objective of this exercise is to solve the TSP problem for a set of 14 cities in Burma, officially the Republic of the Union of Myanmar. The following vectors give the GPS position of each city.

   LAT = [16.47, 16.47, 20.09, 22.39, 25.23, 22.00, 20.47,
          17.20, 16.30, 14.05, 16.53, 21.52, 19.41, 20.09]

   LON = [96.10, 94.44, 92.54, 93.37, 97.24, 96.05, 97.02,
          96.29, 97.38, 98.12, 97.38, 95.59, 97.13, 94.55]
  
Consider that the distance between two cities is not the Euclidean distance. You have to consider that they are points on the surface of the Earth, which can be considered to be an oblate spheroid.

In this application, we are interested in getting the best solution as possible (e.g., the shortest path). However, notice that we do not know the optimal solution beforehand, so you have to try the GA several times to check if there is a better solution. In this experiment we are not evaluating the capabilities of the Genetic Algorithm, so you do not have to test it using different parameters (mutation and crossover rates, population, etc) to show this, but you may need to try different parameters to be able to find a better solution to your problem.I hope this is clear sourire

## Install libraries and imports

Vincenty library is used to calculate the distance between to GPS point on earth. [Vincenty link](https://pypi.org/project/vincenty/)

In [1]:
!pip install vincenty

  Using cached vincenty-0.1.4.tar.gz (2.8 kB)
  Created wheel for vincenty: filename=vincenty-0.1.4-py3-none-any.whl size=3080 sha256=319072a30c98e64fe63114f8e962eeec5ceb5214bcc2ac8ed96cc22dd2ada696
  Stored in directory: c:\users\alecb\appdata\local\pip\cache\wheels\2f\4d\df\6e2ce31c63c93508b38ed9098c1483e7616b55a6ddabcd2f5b
Successfully built vincenty


In [2]:
from pyevolve import G1DList
from pyevolve import GSimpleGA
from pyevolve import Selectors
from pyevolve import Statistics
from vincenty import vincenty

## Defining datas
Indexes is used as city ID

In [4]:
cities_latitude = [16.47, 16.47, 20.09, 22.39, 25.23, 22.00, 20.47, 
          17.20, 16.30, 14.05, 16.53, 21.52, 19.41, 20.09]
cities_longitude = [96.10, 94.44, 92.54, 93.37, 97.24, 96.05, 97.02, 
          96.29, 97.38, 98.12, 97.38, 95.59, 97.13, 94.55]

coordinates = list(zip(cities_latitude, cities_longitude))

 ## Defining function

In [6]:
"""
Calculate the distance between two lat, long coordinates.

Parameters
----------
`origin` : tuple of float (lat, long)
`dest`   : tuple of float (lat, long)

Returns
-------
distance_in_km : float
"""
def distance(origin, destination):
    return vincenty(origin, destination)

In [7]:
"""
Calculate the distances between all the towns in a list

Parameters
----------
`coord` : List of tuples (lat, long) representing towns

Returns
-------
Matrix containing the distances between each town
"""
def distance_matrix(coordinates):
    matrix = {}
    for i, origin in enumerate(coordinates):
        for j, dest in enumerate(coordinates):
            matrix[i, j] = distance(origin, dest)
    return matrix

## Defining Fitness

In [ ]:
"""
Fitness function for the genetic algorithm

Parameters
----------
`genome` : the genome to fit

Returns
-------
The score of the given genome
"""
def fitness(genome):
    global distances
    
    score = 0.0

    for i in range(len(genome)):
        # get index of the next city
        # note: when the src city is the last one in
        #       the chromosome, the next one is the first one.
        #       hence the modulo
        j = (i + 1) % len(genome)
        score += distances[genome[i], genome[j]]
    return score

In [3]:
def fitness(chromosome):
    score = 0.0
    values = []
    # iterate over the chromosome
    for i in range(0,len(chromosome),4):
        value = chromosome[i+3]+chromosome[i+2]*2+chromosome[i+1]*4+chromosome[i]*8
        if (value == 0):
            value = 16
        values.append(value)
    score = (values[0]*values[1]*values[2]*values[3])/(values[4]*values[5]*values[6]*values[7])
    return score

# The evaluation function
#global cm
#return tour_length(cm, chromosome)

#def tour_length(matrix, tour):
   #Returns the total length of the tour
   #total=0
   #num_cities=len(tour)
   #for i in range(num_cities):
      #j=(i+1)%num_cities
      #city_i=tour[i]
      #city_j=tour[j]
      #total+=matrix[city_i,city_j]
   #return total

In [ ]:
def cartesian_matrix(coords):
   """ A distance matrix """
   matrix={}
   for i,(x1,y1) in enumerate(coords):
      for j,(x2,y2) in enumerate(coords):
         dx,dy=x1-x2,y1-y2
         dist=sqrt(dx*dx + dy*dy)
         matrix[i,j]=dist
   return matrix

In [ ]:
def tour_length(matrix, tour):
   """ Returns the total length of the tour """
   total=0
   num_cities=len(tour)
   for i in range(num_cities):
      j=(i+1)%num_cities
      city_i=tour[i]
      city_j=tour[j]
      total+=matrix[city_i,city_j]
   return total

In [ ]:
def G1DListTSPInitializator(genome, **args):
   """ The initializator for the TSP """
   genome.clearList()
   lst = [i for i in xrange(genome.listSize)]

   for i in xrange(genome.listSize):
      choice = random.choice(lst)
      lst.remove(choice)
      genome.append(choice)

cm = []
coords = []

In [ ]:
def eval_func(chromosome):
   """ The evaluation function """
   global cm
   return tour_length(cm, chromosome)

## Train

In [ ]:
def init_tsp(genome, **kwargs):
    genome.clearList()
    lst = [i for i in range(genome.getListSize())]
    random.shuffle(lst)

    for city in lst:
        genome.append(city)

In [3]:
# Chromosome representation

# genome = bitstring
genome = G1DBinaryString.G1DBinaryString(32)

# how to compute the fitness
genome.evaluator.set(fitness)

# From tsp example
#genome.evaluator.set(eval_func)
#genome.mutator.set(Mutators.G1DListMutatorSwap)
#genome.crossover.set(Crossovers.G1DListCrossoverOX)
#genome.initializator.set(G1DListTSPInitializator)

# GA initialisation
ga = GSimpleGA.GSimpleGA(genome, seed=999)

# From tsp example
#ga.setGenerations(1000)
#ga.setMinimax(Consts.minimaxType["minimize"])
#ga.setCrossoverRate(1.0)
#ga.setMutationRate(0.03)
#ga.setPopulationSize(80)

ga.setPopulationSize(100)
ga.setMutationRate(0.01)
ga.setCrossoverRate(0.9)
ga.selector.set(Selectors.GTournamentSelector)
ga.setElitism(True)

# Number of generations
ga.setGenerations(100)

# In case we want to monitor the evolution process
# execute the function current_best every generation
#ga.stepCallback.set(current_best)

ga.evolve(freq_stats=10)

# From tsp example
#ga.evolve(freq_stats=100)
#best = ga.bestIndividual()

# Final best solution
print(ga.bestIndividual())

Gen. 0 (0.00%): Max/Min/Avg Fitness(Raw)             [15.45(720.00)/12.82(0.00)/12.87(12.87)]
Gen. 10 (10.00%): Max/Min/Avg Fitness(Raw)             [10996.23(26880.00)/8219.45(37.33)/9163.52(9163.52)]
Gen. 20 (20.00%): Max/Min/Avg Fitness(Raw)             [24040.06(30720.00)/12662.97(375.00)/20033.38(20033.38)]
Gen. 30 (30.00%): Max/Min/Avg Fitness(Raw)             [33080.42(32768.00)/0.00(1024.00)/25347.13(25347.13)]
Gen. 40 (40.00%): Max/Min/Avg Fitness(Raw)             [33031.30(32768.00)/0.00(682.67)/23837.35(23837.35)]
Gen. 50 (50.00%): Max/Min/Avg Fitness(Raw)             [32832.23(32768.00)/0.00(256.00)/26246.54(26246.54)]
Gen. 60 (60.00%): Max/Min/Avg Fitness(Raw)             [32863.03(32768.00)/0.00(512.00)/27718.20(27718.20)]
Gen. 70 (70.00%): Max/Min/Avg Fitness(Raw)             [32920.51(32768.00)/0.00(409.60)/23988.45(23988.45)]
Gen. 80 (80.00%): Max/Min/Avg Fitness(Raw)             [32805.49(32768.00)/0.00(102.40)/24014.47(24014.47)]
Gen. 90 (90.00%): Max/Min/Avg Fitness